In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


Так как модель работы сайта и процесс конвертации представляют досточно прямолинейную модель без большого количество взаимозависимых и влияющих друг на друга переменных, для построения модели выбрал линейную регрессию. Однако, её применения осложнялось большим количеством категорийных данных. Для решения этой проблемы использовал one-hot encoding.
   

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [4]:
!mkdir train

mkdir: cannot create directory ‘train’: File exists


In [5]:
!mkdir test

In [6]:
!wget -O /content/train/train.csv "https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_converted_users.csv"

--2022-09-04 14:08:50--  https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_converted_users.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.217.212.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219241 (214K) [application/octet-stream]
Saving to: ‘/content/train/train.csv’

/content/train/trai 100%[===================>] 214.10K  --.-KB/s    in 0.002s  

2022-09-04 14:08:50 (92.6 MB/s) - ‘/content/train/train.csv’ saved [219241/219241]



In [7]:
!wget -O /content/test/test_0.csv "https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000000.csv"
!wget -O /content/test/test_1.csv "https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000001.csv"
!wget -O /content/test/test_2.csv "https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000002.csv"
!wget -O /content/test/test_3.csv "https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000003.csv"
!wget -O /content/test/test_4.csv "https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000004.csv"

--2022-09-04 14:08:56--  https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000000.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.217.212.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208104349 (198M) [application/octet-stream]
Saving to: ‘/content/test/test_0.csv’

/content/test/test_ 100%[===================>] 198.46M   188MB/s    in 1.1s    

2022-09-04 14:08:57 (188 MB/s) - ‘/content/test/test_0.csv’ saved [208104349/208104349]

--2022-09-04 14:08:57--  https://storage.googleapis.com/9e97i3wgzessnidlvx75t/pwc_task_demo_dataset_000000000001.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.128, 172.217.212.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.128|:443... connected.
HTTP request sent, awaiting resp

In [8]:
df_test = pd.read_csv("./test/test_0.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
for i in range(1, 2):
  df_test = pd.concat([df_test, pd.read_csv(f'./test/test_{i}.csv')])

Изначально планировал соездинить все части дата-сета, однако мощности ноутбука не хватало для вычислений, в связи с этим соединил только 2 части для тестирования.

In [10]:
len(df_test)

1660006

In [11]:
df_train = pd.read_csv('./train/train.csv')

In [12]:
df_merged = pd.merge(df_test, right=df_train, left_on='fullVisitorId', right_on='fullvisitorId', how='left').drop('fullvisitorId', axis=1)

In [13]:
del df_test
del df_train

In [14]:
df_merged['converted'] = df_merged['converted'].fillna(0)

In [15]:
df_merged['date'] = pd.to_datetime(df_merged['date'], format='%Y-%m-%d')

In [16]:
df_merged['referralPath'] = df_merged['referralPath'].str.split('/', 2).str[1]

In [17]:
df_merged['pagePath'] = df_merged['pagePath'].str.split('/', 2).str[1].value_counts()

Также для оптимизации модели были соединены некоторые значения в столбцах referalPath, pagePath. Например, все подвиды источника "youtube" сокращались для одного, так как one hot encoding создается дополнительные столбы для уникальных значений.


In [10]:
X = df_merged.drop(['fullVisitorId', 'sessionId', 'date', 'city', 'eventLabel', 'converted'], axis=1)
Y = df_merged['converted']

Выше указанные столбцы также были отброшены из модели ввиду большого количества уникальных значений с допущением, что их потеря не сильно определяет поведение модели. 


In [19]:
test_size = 0.25

In [23]:
df_merged[["referralPath", "campaign", "source", "medium", "browser", "operatingSystem", "deviceCategory", "country", "region", "eventCategory", "eventAction", "eventLabel", "channelGrouping", "pagePath", "type"]].apply(pd.Series.nunique)

referralPath       299
campaign             8
source             245
medium               7
browser             45
operatingSystem     19
deviceCategory       3
country            213
region             376
eventCategory        2
eventAction          6
eventLabel         488
channelGrouping      8
pagePath             0
type                 2
dtype: int64

In [24]:
df = df_merged.copy()
del df_merged

In [25]:
df.columns

Index(['fullVisitorId', 'sessionId', 'date', 'visits', 'hits', 'pageviews',
       'timeOnSite', 'bounces', 'newVisits', 'screenviews',
       'uniqueScreenviews', 'timeOnScreen', 'referralPath', 'campaign',
       'source', 'medium', 'browser', 'operatingSystem', 'isMobile',
       'deviceCategory', 'country', 'region', 'city', 'hitNumber',
       'eventCategory', 'eventAction', 'eventLabel', 'channelGrouping',
       'action_type', 'pagePath', 'type', 'converted'],
      dtype='object')

In [26]:
ONE_HOT_COLS = ["campaign", "medium", "browser", "operatingSystem", "deviceCategory", "eventCategory", "eventAction", "channelGrouping", "type"]
print("Starting DF shape: %d, %d" % df.shape)


for col in ONE_HOT_COLS:
    s = df[col].unique()

    # Create a One Hot Dataframe with 1 row for each unique value
    one_hot_df = pd.get_dummies(s, prefix='%s_' % col)
    one_hot_df[col] = s

    print("Adding One Hot values for %s (the column has %d unique values)" % (col, len(s)))
    pre_len = len(df)

    # Merge the one hot columns
    df = df.merge(one_hot_df, on=[col], how="left")
    assert len(df) == pre_len
    print(df.shape)

Starting DF shape: 1660006, 32
Adding One Hot values for campaign (the column has 8 unique values)
(1660006, 40)
Adding One Hot values for medium (the column has 7 unique values)
(1660006, 47)
Adding One Hot values for browser (the column has 45 unique values)
(1660006, 92)
Adding One Hot values for operatingSystem (the column has 19 unique values)
(1660006, 111)
Adding One Hot values for deviceCategory (the column has 3 unique values)
(1660006, 114)
Adding One Hot values for eventCategory (the column has 3 unique values)
(1660006, 116)
Adding One Hot values for eventAction (the column has 7 unique values)
(1660006, 122)
Adding One Hot values for channelGrouping (the column has 8 unique values)
(1660006, 130)
Adding One Hot values for type (the column has 2 unique values)
(1660006, 132)


In [27]:
X = df.drop(['fullVisitorId', 'sessionId', 'date', 'source', 'referralPath', 'country', 'region', 'city', 'eventLabel', 'pagePath', 'converted'], axis=1)
X = X.drop(ONE_HOT_COLS, axis=1)
Y = df['converted']

del df

In [28]:
test_size = 0.25

В качестве выборки выбрал 1/4 генеральной совокупности

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

In [30]:
len(X_train.columns)

112

In [32]:
from sklearn.ensemble import RandomForestClassifier

В качестве основного алгоритма выбрал множество решающих деревьев ввиду его универсальности. Так эту задачу можно отнести как к классифицкации, так и к кластеризации, то random forest здесь будет наиболее полезен.

In [33]:
rf_model = RandomForestClassifier(n_estimators=20, max_features="auto")

In [34]:
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)

In [35]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20)

In [36]:
X_test = X_test.fillna(0)

In [37]:
predictions = rf_model.predict(X_test)

In [38]:
accuracy_score(y_test, predictions)

0.9320918935330432

Точность модели составила 93.2%, для большего понимания предсказания ниже вывел основные параметры влияющие на конвертацию пользователей.


In [39]:
y_test.iloc[0]

0.0

In [40]:
for i in range(20):
  print(predictions[i], y_test.iloc[i])

0.0 0.0
0.0 0.0
0.0 0.0
1.0 1.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
1.0 1.0
1.0 1.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
1.0 1.0
1.0 1.0


In [41]:
importances = rf_model.feature_importances_
columns = X_test.columns

Из приведенных ниже данных видно, что наибольший коэффициент составляет для следующих показателей: время (в секундах), проведенное за сессю (30.3%) количество просмотренных страниц за сессию (17,2%), количество кликов за сессию (17,8%) и другие. Если сделать некоторые допущения и смоделировать некоторый процесс покупки/ознакомления с интернетом-магазином, основываясь на своём опыте, то выведенная модель и её ключевые значения выглядят достаточно реалистично и могут быть применимы на практике.

In [42]:
for i in range(len(columns)):
  print(columns[i], '{0:f}'.format(importances[i]))

visits 0.000000
hits 0.178912
pageviews 0.172843
timeOnSite 0.303609
bounces 0.000413
newVisits 0.048939
screenviews 0.000000
uniqueScreenviews 0.000000
timeOnScreen 0.000000
isMobile 0.004766
hitNumber 0.116567
action_type 0.050183
campaign__(not set) 0.000717
campaign__AW - Accessories 0.000486
campaign__AW - Apparel 0.000024
campaign__AW - Dynamic Search Ads Whole Site 0.000595
campaign__AW - Electronics 0.000105
campaign__All Products 0.000000
campaign__Data Share 0.000000
campaign__Data Share Promo 0.000526
medium__(none) 0.009396
medium__(not set) 0.000031
medium__affiliate 0.000452
medium__cpc 0.000587
medium__cpm 0.000882
medium__organic 0.002689
medium__referral 0.006424
browser__(not set) 0.000000
browser__0 0.000000
browser__ADM 0.000000
browser__Amazon Silk 0.000022
browser__Android Browser 0.000001
browser__Android Webview 0.000138
browser__Apple-iPhone7C2 0.000000
browser__BlackBerry 0.000001
browser__Chrome 0.005619
browser__Coc Coc 0.000028
browser__DASH_JR_3G 0.000000
